In [ ]:
# @title ## **1. Install HealthINU Trainer**
# @markdown ##### 별거 없으면 그냥 시작 버튼만 눌러도 됨
import os
import shutil
import time
import requests
import torch
from subprocess import getoutput
from IPython.display import clear_output
from google.colab import drive
import ipywidgets as widgets
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

%store -r

# root_dir
root_dir          = "/content"
drive_dir         = os.path.join(root_dir, "drive/MyDrive")
repo_dir          = os.path.join(root_dir, "HealthINU-img")
training_dir      = os.path.join(repo_dir, "dataset")
log_dir           = os.path.join(repo_dir, "log")

repository        = "Default" #@param ["Default"] {allow-input: false}
repo_url          = "https://github.com/HealthINU/HealthINU-img"
branch            = "main"  # @param {type: "string"}
output_to_drive   = True  # @param {type: "boolean"}

def clone_repo(url, dir, branch):
    if not os.path.exists(dir):
       !git clone -b {branch} {url} {dir}

def mount_drive(dir, name):
    output_dir      = os.path.join(training_dir, name)
    if output_to_drive:
        if not os.path.exists(drive_dir):
            drive.mount(os.path.dirname(drive_dir))
        output_dir  = os.path.join(drive_dir, "HealthINU-img/")
        output_dir  = os.path.join(output_dir, name)
    return output_dir

def setup_directories():
    global output_dir
    global log_dir

    output_dir = mount_drive(drive_dir,"output")
    log_dir = mount_drive(drive_dir,"log")

    for dir in [log_dir, output_dir]:
        os.makedirs(dir, exist_ok=True)

def install_dependencies():
    requirements_file = os.path.join(repo_dir, "requirements.txt")
    gpu_info          = getoutput('nvidia-smi')

    !pip install -q --upgrade -r {requirements_file}

def main():
    os.chdir(root_dir)
    clone_repo(repo_url, repo_dir, branch)
    os.chdir(repo_dir)
    setup_directories()
    install_dependencies()

main()

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
# @title ## **1-1. Unzip Dataset (Optional)**
import zipfile

# @markdown #### 참고로 구글 드라이브 경로는 //content/drive/MyDrive/ 부터 시작됨
# @markdown ##### 대충 MyDrive 폴더 내부가 구글 드라이브란 이야기
# 압축을 해제할 '/파일경로/파일명.zip'
zip_path            = "/content/drive/MyDrive/HealthINU-img/dataset.zip"  # @param {type: "string"}

# 압축을 해제할 위치경로
zip = zipfile.ZipFile(zip_path)
zip.extractall(training_dir)

inf('\u2714 Done','success', '50px')

In [ ]:
# @title ## **2. Run HealthINU Trainer**
# @markdown ### 학습 설정
epochs            = "50"  # @param {type: "string"}
learning_rate     = "0.0001"  # @param {type: "string"}
batch_size        = "32"  # @param {type: "string"}
num_workers       = "2"  # @param {type: "string"}
early_patience       = "15"  # @param {type: "string"}
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;※&#160;early_patience는 val_loss가 계속 증가하지 않을 때의 용납 가능한 횟수
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;값이 0일경우 측정 안함
optimizer         = "Adam" #@param ["Adam", "AdamW", "SGD"] {allow-input: false}
weight_decay          = "0"  # @param {type: "string"}
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;※&#160;weight_decay 값은 모델의 가중치 값이 지나치게 커지는 것을 방지하기 위해 가중치에 패널티를 부과
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;Adam, AdamW의 경우에는 보통 0\~0.01, &#160;&#160;SGD는 0\~0.0005 권장이라곤 하는데 일단 써봐야 알 듯
isFixedSeed = False  # @param {type: "boolean"}
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;※&#160;Seed 고정 여부
isFreeze   = False  # @param {type: "boolean"}
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;※&#160;isFreeze 변수는 True 일 경우, 특징 추출 파트를 Freeze시켜서 분류부분만 학습하도록 함
# @markdown ##### &#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;근데 얘는 끄는게 맞는 것 같음
# 학습함수, 테스트함수 불러오기
from img_train import train_set, validate_image
from img_predict import ImageClassifier

print("이미지 학습 시작")
print(training_dir)
train_set(training_dir, True, int(epochs), float(learning_rate), int(batch_size), int(num_workers), int(early_patience), float(weight_decay), optimizer, isFixedSeed, isFreeze)

# 학습 후 현재시간 가져옴
from datetime import datetime
from pytz import timezone
s = datetime.now(timezone('Asia/Seoul')).strftime("model-pretrained_%Y%m%d_%H%M%S.pth")
s = os.path.join(output_dir, s)

# 현재시간 가져옴 2
c = datetime.now(timezone('Asia/Seoul')).strftime("img_classes_%Y%m%d_%H%M%S.txt")
c = os.path.join(log_dir, s)

# 현재 model-pretrained.pth 모델을 복사해서 s로 된 이름으로 저장
model_dir = os.path.join(repo_dir, "model-pretrained.pth")
log_temp = os.path.join(repo_dir, 'log')
c_temp = os.path.join(repo_dir, 'img_classes.txt')

# log 파일 모두 복사
shutil.copytree(log_temp, log_dir, dirs_exist_ok=True)

# 모델 파일 복사
shutil.copy(model_dir, s)
print("Model saved on your Google Drive")

#
shutil.copy(c_temp, s)
print("Label file saved on your Google Drive")

inf('\u2714 Done','success', '50px')

In [ ]:
# @title ## **2-1. 중간 파일 옮기기**
# @markdown #### 중간에 끊었을 경우 모델, 로그파일을 옮기고 싶으면 클릭
# 학습 후 현재시간 가져옴

from datetime import datetime
from pytz import timezone
s = datetime.now(timezone('Asia/Seoul')).strftime("model-pretrained_%Y%m%d_%H%M%S.pth")
s = os.path.join(output_dir, s)

# 현재시간 가져옴 2
c = datetime.now(timezone('Asia/Seoul')).strftime("img_classes_%Y%m%d_%H%M%S.txt")
c = os.path.join(log_dir, s)

# 현재 model-pretrained.pth 모델을 복사해서 s로 된 이름으로 저장
model_dir = os.path.join(repo_dir, "model-pretrained.pth")
log_temp = os.path.join(repo_dir, 'log')
c_temp = os.path.join(repo_dir, 'img_classes.txt')

# log 파일 모두 복사
shutil.copytree(log_temp, log_dir, dirs_exist_ok=True)

# 모델 파일 복사
shutil.copy(model_dir, s)
print("Model saved on your Google Drive")

# 라벨 파일 복사
shutil.copy(c_temp, s)
print("Label file saved on your Google Drive")

inf('\u2714 Done','success', '50px')

In [ ]:
# @title ## **2-2. GPU VRAM 비우기**
# @markdown #### 이상하게 자꾸 CUDA Out of memory 뜨면 이거 눌러보셈
torch.cuda.empty_cache()

In [ ]:
# @title ## **3. Test Model (Optional)**
# @markdown ##### 야매 테스트임
# ImageClassifier 클래스 인스턴스 생성

import json
from img_predict import ImageClassifier
classifier = ImageClassifier()

correct=0
wrong=0

# test_answer.json 파일 불러오기
with open('test_answer.json') as f:
  test_answer = json.load(f)

# json에 대한 반복문 수행
for key in test_answer:
  # 이미지 검증
  if not(validate_image(key['name'])):
    print("이미지 문제 발생")
    continue

  # 해당 파일에 대한 예측 수행
  result = classifier.predict(key['name'])

  # 예측 결과 출력
  print("---" * 12)
  print("[", key['name'], "]")
  print("예측 : ", list(result.keys())[0], ", 정답 : ", key['answer'])
  if(key['answer'] == list(result.keys())[0]):
    print("결과 : 일치")
    correct+=1
  else:
    print("결과 : 불일치!!")
    wrong+=1
  print("---" * 12)

# 정답률 및 개수 출력
print("---" * 12   )
print("전체 개수 : ", correct + wrong, "개")
print("정답 : ", correct, "개")
print("오답 : ", wrong, "개")
print("정답률 : ", correct / (correct+wrong) * 100, "%")
print("---" * 12   )

In [ ]:
# @title ## **3-1. Test on Specific Image (Optional)**
# @markdown ##### 대충 경로 입력해서 따로 써보는 기능
image_path            = "/content/HealthINU-img/test21.jpg"  # @param {type: "string"}

# 이미지 오염 여부 확인
if(validate_image(image_path)):
  print("이미지 분류 시작")
  # ImageClassifier 클래스 인스턴스 생성
  classifier = ImageClassifier()

  # 이미지 파일에 대한 예측 수행
  result = classifier.predict(image_path)
  print(result)
else: print("이미지 문제 발생")


In [ ]:
# @title ## **4. Colab Unassign**
# @markdown #### 쓸거 다 썼으면 코랩 할당 해제 하기
# @markdown #### 굳이 이거 아니어도 오른쪽 위에 '런타임 연결 해제 및 삭제' 눌러도 됨
from google.colab import runtime
runtime.unassign()